### Transport modeling based on a flow model

In [ ]:
#Basic lines for transport modeling
import flopy ## Org
import json, os ## Org
import numpy as np ## Org
import geopandas as gpd ## Org
import matplotlib.pyplot as plt ## Org
from mf6Voronoi.tools.cellWork import getLayCellElevTupleFromRaster, getLayCellElevTupleFromObs ## Org

In [ ]:
# load simulation
simName = 'mf6Sim' ## Org
modelName = 'mf6Model' ## Org 
modelWs = os.path.abspath('../modelFiles') ## Org
sim = flopy.mf6.MFSimulation.load(sim_name=modelName, version='mf6', ## Org
                             exe_name='../../Bin/mf6.exe', ## Org
                             sim_ws=modelWs) ## Org
transWs  = os.path.abspath('../transFiles') ## Org
#change working directory
sim.set_sim_path(transWs) ## Org
sim.write_simulation(silent=True) ## Org

In [ ]:
#list model names
sim.model_names ## Org

In [ ]:
#select the flow model
gwf = sim.get_model('mf6model') ## Org

In [ ]:
# open the json file
with open('../json/disvDict.json') as file: ## Org
    gridProps = json.load(file) ## Org

cell2d = gridProps['cell2d']           #cellid, cell centroid xy, vertex number and vertex id list ## Org
vertices = gridProps['vertices']       #vertex id and xy coordinates ## Org
ncpl = gridProps['ncpl']               #number of cells per layer ## Org
nvert = gridProps['nvert']             #number of verts ## Org
centroids=gridProps['centroids']       ## Org

In [ ]:
#define the transport model ## Org
gwt = flopy.mf6.ModflowGwt(sim, ## Org
                           modelname='gwtModel', ## Org
                           save_flows=True) ## Org

In [ ]:
#register solver for transport model
imsGwt = flopy.mf6.ModflowIms(sim, print_option='SUMMARY', ## Org 
                              outer_dvclose=1e-4, ## Org
                              inner_dvclose=1e-4, ## Org
                              linear_acceleration='BICGSTAB') ## Org
sim.register_ims_package(imsGwt,[gwt.name]) ## Org

In [ ]:
# apply discretization to transport model
disv = flopy.mf6.ModflowGwtdisv(gwt,  ## Org 
                                nlay=gwf.modelgrid.nlay,  ## Org 
                                ncpl=ncpl,  ## Org 
                                top=gwf.modelgrid.top,  ## Org 
                                botm=gwf.modelgrid.botm,  ## Org 
                                nvert=nvert,  ## Org 
                                vertices=vertices,  ## Org 
                                cell2d=cell2d)  ## Org 

In [ ]:
#define starting concentrations
strtConc = np.zeros((gwf.modelgrid.nlay, ncpl), dtype=np.float32) ## Org 

interIx = flopy.utils.gridintersect.GridIntersect(gwf.modelgrid) ## Org 

gwtIc = flopy.mf6.ModflowGwtic(gwt, strt=strtConc) ## Org 

In [ ]:
fig = plt.figure(figsize=(12, 12)) ## Org 
ax = fig.add_subplot(1, 1, 1, aspect = 'equal') ## Org 
mapview = flopy.plot.PlotMapView(model=gwf,layer = 2) ## Org 

plot_array = mapview.plot_array(strtConc,masked_values=[-1e+30], cmap=plt.cm.summer) ## Org 
plt.colorbar(plot_array, shrink=0.75,orientation='horizontal', pad=0.08, aspect=50) ## Org 

In [ ]:
# set advection, dispersion
adv = flopy.mf6.ModflowGwtadv(gwt, scheme='UPSTREAM') ## Org 
dsp = flopy.mf6.ModflowGwtdsp(gwt, alh=1.0, ath1=0.1) ## Org 

#define mobile storage and transfer
porosity = 0.05 ## Org 
sto = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity) ## Org 

In [ ]:
#define sink and source package
sourcerecarray = [()] ## Org 
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray) ## Org 

In [ ]:
cncSpd = {} ## Org
cncSpd[0] = [] ## Org
cnc = flopy.mf6.ModflowGwtcnc(gwt,stress_period_data=cncSpd) ## Org
cnc.plot(mflay=2) ## Org

In [ ]:
obsList = []
nameList, obsLayCellList = getLayCellElevTupleFromObs(gwf, ## Org
                  interIx, ## Org
                  '../shp/##piezometers##.shp', ## Org
                  '##nameField##', ## Org
                  '##elevField##') ## Org

for obsName, obsLayCell in zip(nameList, obsLayCellList): ## Org
    obsList.append((obsName,'concentration',obsLayCell[0]+1,obsLayCell[1]+1)) ## Org


obs = flopy.mf6.ModflowUtlobs( ## Org
    gwt,
    filename=gwt.name+'.obs', ## Org
    digits=10, ## Org
    print_input=True, ## Org
    continuous={gwt.name+'.obs.csv': obsList} ## Org
)

In [ ]:
#define output control
oc_gwt = flopy.mf6.ModflowGwtoc(gwt, ## Org
                                budget_filerecord='%s.cbc'%gwt.name, ## Org
                                concentration_filerecord='%s.ucn'%gwt.name, ## Org
                                saverecord=[('CONCENTRATION', 'LAST'), ('BUDGET', 'LAST')], ## Org
                                printrecord=[('CONCENTRATION', 'LAST'), ('BUDGET', 'LAST')], ## Org
                                )

In [ ]:
pd = [ ## Org
    ("GWFHEAD", "../modelFiles/##headFile##.hds"), ## Org
    ("GWFBUDGET", "../modelFiles/##budgetFile##.cbc"), ## Org
]

fmi = flopy.mf6.ModflowGwtfmi(gwt, ## Org
                              flow_imbalance_correction=True, ## Org
                              packagedata=pd) ## Org

In [ ]:
sim.write_simulation() ## Org
success, buff = sim.run_simulation() ## Org

In [ ]:
concObj = gwt.output.concentration() ## Org
concObj.get_times() ## Org

In [ ]:
conc = concObj.get_data() ## Org
conc.shape ## Org

In [ ]:
transAoi = gpd.read_file('../shp/##soluteRefinement##.shp') ## Org
xMin, yMin, xMax, yMax = transAoi.bounds.iloc[0].values ## Org

In [ ]:
mflay = 2 ## Org

# Plot the heads for a defined layer and boundary conditions
fig = plt.figure(figsize=(12,8))  ## Org
ax = fig.add_subplot(1, 1, 1, aspect='equal')  ## Org
modelmap = flopy.plot.PlotMapView(model=gwf)  ## Org

levels = np.linspace(0,conc.max()/2,num=10)  ## Org
quadmesh = modelmap.plot_bc('DRN', color='crimson')  ## Org

contour = modelmap.contour_array(conc[mflay],ax=ax,levels=levels,cmap='summer')  ## Org
ax.clabel(contour)  ## Org

linecollection = modelmap.plot_grid(linewidth=0.1, alpha=0.8, color='cyan', ax=ax)  ## Org

cellConc = modelmap.plot_array(conc[mflay],ax=ax,cmap='Blues')  ## Org

quadmesh = modelmap.plot_bc('DRN', color='slategrey')  ## Org

#dump1 = modelmap.plot_shapefile('../shp/wasteDump1.shp')
#piezo = modelmap.plot_shapefile('../shp/piezometers2.shp', radius=10)

ax.set_xlim(xMin,xMax)
ax.set_ylim(yMin,yMax)

plt.colorbar(cellConc, shrink=0.25) 

plt.show() 